Only made to test numpy array

In [1]:
import re
import os
import json
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from collections import defaultdict

# Merging data from NKRYa into one json

In [5]:
words_num = 99
data_files_format = 'data/RuShiftEval/test/eval_23_True_True/2-3_{0}.data'

In [6]:
def load_words(data_files_format, words_num=99):
    all_words = []
    
    for word_id in range(words_num):
        current_file = data_files_format.format(word_id)
        with open(current_file) as f:
            word_examples = json.load(f)
            all_words.extend(word_examples)
            
    return all_words

In [7]:
all_words = load_words(data_files_format, words_num=words_num)

len(all_words)

9149

In [46]:
all_words[200]

{'id': 'test.scd_2.31',
 'lemma': 'апостол',
 'pos': 'NOUN',
 'sentence1': 'Ложные пророки ее и апостолы -- Юнг-Штиллинг, Эккартсгаузен, Гион, Беме, Лабзин, Госнер, Фесслер, методисты, гернгутеры"...',
 'sentence2': 'Муки совести, трагедийное чувство вины приобретают у Петрухи едва ли не христианский характер и отделяют его от остальных "апостолов" нового мира, идущих "без имени святого".',
 'start1': 20,
 'end1': 28,
 'start2': 123,
 'end2': 132,
 'grp': 'COMPARE'}

In [47]:
with open('data/RuShiftEval/test/eval_23.json', 'w', encoding='utf-8') as f:
    json.dump(all_words, f, indent=4, ensure_ascii=False)

# Getting average score per word

In [26]:
preds_filename = 'data/RuShiftEval/RuShiftEvalPreds/base_raw/preds_eval_23.json'
data_files_format = 'data/RuShiftEval/test/eval_23_True_True/2-3_{0}.data'
output_epochs_filename = 'data/RuShiftEval/RuShiftEvalPreds/words_23.json'

In [27]:
all_words = load_words(data_files_format, words_num=words_num)
all_words = {word['id']: word for word in all_words}

len(all_words)

9149

In [28]:
class VectorsDotPredictor:
    def __init__(self, threshold=None, normalize=True, norm_ord=2):
        self.threshold = threshold
        self.normalize = normalize
        self.norm_ord = norm_ord
    
    def predict(self, out_vector_1, out_vector_2):
        return self.predict_proba(out_vector_1, out_vector_2) > self.threshold
    
    def predict_proba(self, out_vector_1, out_vector_2):
        out_vector_1 = np.array(out_vector_1, dtype='float64')
        out_vector_2 = np.array(out_vector_2, dtype='float64')
#         print(out_vector_1)
#         print(out_vector_1 * out_vector_2)
        
        if self.normalize:
            out_vector_1 /= np.linalg.norm(out_vector_1, ord=self.norm_ord)
            out_vector_2 /= np.linalg.norm(out_vector_2, ord=self.norm_ord)
            
        return np.sum(out_vector_1 * out_vector_2)

In [29]:
class VectorsDistPredictor:
    def __init__(self, threshold, normalize=True, norm_ord=2):
        self.threshold = threshold
        self.normalize = normalize
        self.norm_ord = norm_ord
    
    def predict(self, out_vector_1, out_vector_2):
        return self.predict_proba(out_vector_1, out_vector_2) < self.threshold
    
    def predict_proba(self, out_vector_1, out_vector_2):
        out_vector_1 = np.array(out_vector_1)
        out_vector_2 = np.array(out_vector_2)
        
        if self.normalize:
            out_vector_1 /= np.linalg.norm(out_vector_1, ord=self.norm_ord)
            out_vector_2 /= np.linalg.norm(out_vector_2, ord=self.norm_ord)
        
        return np.linalg.norm(out_vector_1 - out_vector_2, ord=self.norm_ord)

In [30]:
with open(preds_filename) as f:
    eval_samples = json.load(f)

In [31]:
word_to_dists = defaultdict(list)
# predictor = VectorsDistPredictor(threshold=None, normalize=True, norm_ord=1)
# predictor = VectorsDistPredictor(threshold=None, normalize=True, norm_ord=2)
predictor = VectorsDotPredictor(threshold=None, normalize=False)

for sample in eval_samples:
    dist = predictor.predict_proba(sample['context_output1'], sample['context_output2'])
    current_lemma = all_words[sample['id']]['lemma']
    word_to_dists[current_lemma].append(dist)
    
len(word_to_dists)

99

In [32]:
word_to_score = {}

for word, dists in word_to_dists.items():
    word_to_score[word] = np.mean(dists)    
        
len(word_to_score)

99

In [33]:
with open(output_epochs_filename, 'w', encoding='utf-8') as f:
    json.dump(word_to_score, f, indent=4, ensure_ascii=False)

In [34]:
word_to_score

{'авторитет': 366.79812865806343,
 'амбиция': 367.2768907843167,
 'апостол': 369.0906733970125,
 'благодарность': 365.20261136022253,
 'блин': 363.41510445376525,
 'блондин': 370.37098893158674,
 'брат': 368.13799579496975,
 'бригада': 368.29423997031097,
 'веер': 362.9078374413895,
 'век': 367.7791638279774,
 'вызов': 364.20412905834326,
 'головка': 365.30238379794594,
 'грех': 366.09194974951345,
 'дух': 368.10210429092854,
 'дядька': 369.2114265593414,
 'дядя': 370.40724664777485,
 'железо': 364.3310378177398,
 'жесть': 362.77303997994716,
 'живот': 365.4523285382051,
 'заблуждение': 366.99401648342604,
 'издательство': 371.3032522249927,
 'итальянец': 368.59500079573127,
 'кабан': 364.6686926602189,
 'карман': 364.49243658502945,
 'крушение': 366.4311744643492,
 'крыша': 365.9442301309654,
 'кулиса': 362.67314967980616,
 'лечение': 371.56879414880854,
 'линейка': 365.29516780343414,
 'лишение': 367.42099087312204,
 'локоть': 363.2901587669237,
 'любовник': 368.3733397283336,
 'любо

# Geeting final results

In [35]:
output_epochs_12_filename = 'data/RuShiftEval/RuShiftEvalPreds/words_12.json'
output_epochs_13_filename = 'data/RuShiftEval/RuShiftEvalPreds/words_13.json'
output_epochs_23_filename = 'data/RuShiftEval/RuShiftEvalPreds/words_23.json'

prediction_filename = 'data/RuShiftEval/RuShiftEvalPreds/post_preds/prediction_base_mlm_dot_v3.tsv'

In [36]:
def load_scores(filename):
    with open(filename) as f:
        return json.load(f)

In [37]:
scores_12 = load_scores(output_epochs_12_filename)
scores_13 = load_scores(output_epochs_13_filename)
scores_23 = load_scores(output_epochs_23_filename)

len(scores_12), len(scores_13), len(scores_23)

(99, 99, 99)

In [38]:
with open(prediction_filename, 'w', encoding='utf-8') as f:
    for word, score_12 in scores_12.items():
#         score_12 = 1 / score_12
#         score_13 = 1 / scores_13[word]
#         score_23 = 1 / scores_23[word]
        score_12 = score_12
        score_13 = scores_13[word]
        score_23 = scores_23[word]
        f.write(f'{word}\t{score_12}\t{score_23}\t{score_13}\n')